# Day 3

In [1]:
import torch
import torch.nn as nn # building blocks
import torch.nn.functional as F # convolution functions
import torch.optim as optim # various optimization algorithms

from IPython.display import Image

In [2]:
# For reproducibility
torch.manual_seed(1)

### Data

We will use fake data for this example

In [ ]:
x_train = torch.FloatTensor([[1],
                             [2],
                             [3]])
y_train = torch.FloatTensor([[1],
                             [2],
                             [3]])

print(x_train)
print(x_train.shape)
print(y_train)
print(y_train.shape)

기본적으로 Pytorch는 NCHW 형태이다.<br>
tensor는 메모리에 저장할 때 contiguous 형태로 저장되는데 <br>
개수(N), 높이(H), 너비(W), 컬러(C)
NCHW는 채널단위로 각 픽셀의 값이 연속적으로 저장되고 <br> 
NHWC는 픽셀 단위로 채널의 값이 연속적으로 저장된다. <br>

예를 들어, 아래 그림에서 메모리 저장 방식은 다음과 같다. <br>
![Day3](/Volumes/SUHYE/1.research/2024_study/PyTorch/1.Practice/image.png)

- NCHW 방식의 메모리<br>
[0.2 0.4 0.7 0.8 1.5 2.4 6.5 1.2 ... 0.8 0.9 1.4.6.5 ...]<br>
- NHWC 방식의 메모리(tensor)<br>
[0.2 0.8 0.4 0.9 0.7 1.4 0.8 6.5 ....]<br>

tensor 연산은 효율적인 연산을 위해 병렬적으로 수행된다.<br>
따라서 실제 convolution과 같은 연산을 수행할 때 연산에 필요한 값을 벡터화한 후 연산을 수행하게 된다.<br>

#### 장점 <br>
채널 단위로 벡터화하여 연산이 수행되기 때문에 NHWC로 메모리에 저장하는 것이 메모리 엑세스 관점에서 효율적이고, FP16일 때는 메모리 대역폭을 FP32보다 더욱 효율적으로 사용할수 있기 대문에 가속에 큰 장점을 가진다.

### Model

torch.Tenso() 클래스에서 [.requires_grad=True] 를 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(Track) 한다. <br>
계산이 완료된 후 .backward()를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다.
이 Tensor의 변화도는 .grad 속성에 누적된다.

- Tensor가 기록을 추적하는 것을 중단하려면 [.detach()]를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있다.
- 기록을 추적하는 것과 메모리를 사용하는 것을 방지하기 위해, 코드 블럭을 with torch.no_grad()로 감쌀 수 있다.
- 특히 변화도(gradient)는 필요 없지만, [.requires_grad=True]가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evalutate)할 때 유용하다.

In [ ]:
W = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

print('W: ', W)
print('b: ', b)

### Hypothesis

$$ H(x) = Wx + b $$

In [ ]:
hypothesis = x_train * W + b
print('hypothesis: \n', hypothesis)
print('\n')
print('y_train: \n', y_train)

### Cost

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

In [ ]:
print((hypothesis - y_train) ** 2)

In [ ]:
cost = torch.mean((hypothesis - y_train) ** 2)
print('cost: \n', cost)

### Gradient Descent

In [ ]:
optimizer = optim.SGD([W,b], lr=0.01)

optimizer.zero_grad() # gradient 초기화
cost.backward() # gradient 계산
optimizer.step() # 개선된 gradient의 방향대로 W, b 개선


In [ ]:
print('W:\n',W)
print('b: \n',b)

Let's check if the hypothesis is now better.

In [ ]:
hypothesis = x_train * W + b
print('hypothesis: \n', hypothesis)

In [ ]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

### Training with Full Code
In reality, we will be training on the dataset for multiple epochs. <br>
This can be done simply with loops.

In [3]:
# Data
x_train = torch.FloatTensor([[1],
                             [2],
                             [3]])
y_train = torch.FloatTensor([[1],
                             [2],
                             [3]])

# Hypothesis Initialization
hypothesis = x_train * W + b

# Model Initialization
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Optimizer
optimizer = optim.SGD([W,b], lr=0.01)

# Epoch
nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # H(x) 
    hypothesis = x_train * W + b

    # cost
    cost = torch.mean((hypothesis - y_train) ** 2) # square

    # Upgrade H(x) using cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # print log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158
